# Pets Facial Expression classification

## Import the required libraries

In [4]:
import numpy as np 
import pandas as pd

### Read The Image data

In [2]:
TrainingImagePath="C:/Users/SAPTARSHIm/Downloads/Pet's Facial Expression Image Dataset/Master Folder/train"
TestingImagePath="C:/Users/SAPTARSHIm/Downloads/Pet's Facial Expression Image Dataset/Master Folder/test"


In [3]:
from keras.preprocessing.image import ImageDataGenerator
# Defining pre-processing transformations on raw images of training data
train_datagen = ImageDataGenerator(
        rescale=1.0/255,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True)

# Defining pre-processing transformations on raw images of testing data
# test_datagen = ImageDataGenerator(rescale=1.0/255)

# Generating the Training Data
training_set = train_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(128, 128),
        batch_size=16,
        class_mode='categorical')


# Generating the Testing Data
# test_set = test_datagen.flow_from_directory(
#         TrainingImagePath,
#         target_size=(64, 64),
#         batch_size=32,
#         class_mode='categorical')

valid_datagen = ImageDataGenerator(rescale=1.0/255)

valid_set = valid_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(128, 128),
        batch_size=16,
        class_mode='categorical')

# Printing class labels for each face
valid_set.class_indices

Found 1000 images belonging to 4 classes.
Found 1000 images belonging to 4 classes.


{'Angry': 0, 'Other': 1, 'Sad': 2, 'happy': 3}

### Creating a list of facial expression and classes

In [4]:
# class_indices have the numeric tag for each face
TrainClasses=training_set.class_indices

# Storing the face and the numeric tag for future reference
ResultMap={}
for x,emotion in zip(TrainClasses.values(),TrainClasses.keys()):
    ResultMap[x]=emotion

# # Saving the face map for future reference
# import pickle
# with open("ResultsMap.pkl", 'wb') as f:
#     pickle.dump(ResultMap, f, pickle.HIGHEST_PROTOCOL)

print("Mapping of Face and its ID",ResultMap)

Mapping of Face and its ID {0: 'Angry', 1: 'Other', 2: 'Sad', 3: 'happy'}


## Creating the CNN model

In [5]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense


In [6]:
model = Sequential()

# step 1
model.add(Convolution2D(filters=32, kernel_size=(3,3), strides= (1, 1),input_shape=(128,128,3), activation='relu'))

model.add(MaxPool2D(pool_size=(2,2)))

# step 2
model.add(Convolution2D(filters=64, kernel_size=(3,3), strides= (1, 1), activation='relu'))

model.add(MaxPool2D(pool_size=(2,2)))

# step 3

model.add(Convolution2D(filters=64, kernel_size=(4,4), strides= (2, 2), activation='relu'))

model.add(MaxPool2D(pool_size=(2,2)))


model.add(Flatten())

model.add(Dense(64,activation='relu'))

model.add(Dense(len(ResultMap),activation ='softmax'))
#compile the layers
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=["accuracy"])

## Fitting The Model

In [7]:
import time
# Measuring the time taken by the model to train
StartTime=time.time()

# Starting the model training
model.fit(
                    training_set,
                    epochs=50,
                    validation_data=valid_set,
                    validation_steps=2)

EndTime=time.time()
print("############### Total Time Taken: ", round((EndTime-StartTime)/60), 'Minutes #############')

Epoch 1/50
63/63 [==============================] - 20s 278ms/step - loss: 1.3905 - accuracy: 0.2490 - val_loss: 1.3698 - val_accuracy: 0.3125
Epoch 2/50
63/63 [==============================] - 18s 285ms/step - loss: 1.3872 - accuracy: 0.2630 - val_loss: 1.3584 - val_accuracy: 0.2812
Epoch 3/50
63/63 [==============================] - 18s 274ms/step - loss: 1.3779 - accuracy: 0.2940 - val_loss: 1.3593 - val_accuracy: 0.2812
Epoch 4/50
63/63 [==============================] - 18s 279ms/step - loss: 1.3204 - accuracy: 0.3680 - val_loss: 1.2781 - val_accuracy: 0.4062
Epoch 5/50
63/63 [==============================] - 18s 282ms/step - loss: 1.2823 - accuracy: 0.4000 - val_loss: 1.0489 - val_accuracy: 0.6250
Epoch 6/50
63/63 [==============================] - 17s 274ms/step - loss: 1.2328 - accuracy: 0.4460 - val_loss: 1.1363 - val_accuracy: 0.5625
Epoch 7/50
63/63 [==============================] - 18s 278ms/step - loss: 1.1688 - accuracy: 0.4870 - val_loss: 0.9677 - val_accuracy: 0.5625

## saving the model

In [8]:
## Saving the model
model.save("model.pkl")

INFO:tensorflow:Assets written to: model.pkl\assets


INFO:tensorflow:Assets written to: model.pkl\assets


### Testing the model on a different face

In [100]:
'''########################## Making single predictions ############################'''
import numpy as np
from keras.preprocessing import image

testImage="C:/Users/SAPTARSHIm/Downloads/Pet's Facial Expression Image Dataset/Master Folder/test/happy/105.jpg"
test_image=image.load_img(testImage,target_size=(128, 128))
test_image=image.img_to_array(test_image)

test_image_1=np.expand_dims(test_image,axis=0)

result=model.predict(test_image_1,verbose=0)
print(result)
print(ResultMap)

print('####'*10)
print('Prediction is: ',ResultMap[np.argmax(result)])

[[0. 0. 0. 1.]]
{0: 'Angry', 1: 'Other', 2: 'Sad', 3: 'happy'}
########################################
Prediction is:  happy
